In [1]:
!pip install inaSpeechSegmenter ShazamAPI
#have you gone through the trouble of installing CUDA-toolkit and CUDNN?
#!conda install cudnn
#!pip install tensorflow-gpu --upgrade 
#import tensorflow as tf
#print(tf.config.list_physical_devices('GPU'))

ERROR: tensorflow 2.9.1 has requirement numpy>=1.20, but you'll have numpy 1.19.4+mkl which is incompatible.
You should consider upgrading via the 'c:\pthon3.8.5\python.exe -m pip install --upgrade pip' command.



Using legacy setup.py install for docopt, since package 'wheel' is not installed.
Using legacy setup.py install for termcolor, since package 'wheel' is not installed.
    Running setup.py install for docopt: started
    Running setup.py install for docopt: finished with status 'done'
    Running setup.py install for termcolor: started
    Running setup.py install for termcolor: finished with status 'done'


In [1]:
# Load the API
from inaSpeechSegmenter import Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid
import subprocess, os, glob
# select a media to analyse
# any media supported by ffmpeg may be used (video, audio, urls)
import json, shutil
def segment(media, batch_size = 32, energy_ratio = 0.02):
    seg = Segmenter(vad_engine='sm', detect_gender=False, energy_ratio = energy_ratio, batch_size = batch_size)
    segmentation = seg(media)
    return segmentation
def extract_music(segmentation, segment_thres = 60, segment_thres_final = 90, 
                  segment_connect = 3, start_padding = 1, end_padding = 2):
    r = []
    #bridges noEnergy segments that are likely fragmented
    for i in range(len(segmentation)-2, 0, -1):
        if segmentation[i][0] == 'noEnergy' and segmentation[i][2] - segmentation[i][1] < 2 and \
        segmentation[i-1][0] == segmentation[i+1][0]:
            segmentation[i-1] = (segmentation[i-1][0], segmentation[i-1][1], segmentation[i+1][2])
    for i in segmentation: 
        if i[0] == 'music' and i[2]-i[1] > segment_thres: r.append(['',i[1] - start_padding, i[2] + end_padding])    
    for i in range(len(r)-1, 0, -1):
        if r[i][1] - r[i-1][2] < segment_connect:
            r[i-1][2] = r[i][2]
            r[i][1] = r[i][2] + 1    
    rf = []
    for i in r:
        if i[2]-i[1] > segment_thres_final: rf.append(i)
    return [['{}:{}:{}'.format(str(int(x[1]//3600)), 
                               str(int(x[1] % 3600 //60)), 
                               str(int(x[1] % 60)).zfill(2)), 
             '{}:{}:{}'.format(str(int(x[2]//3600)), 
                               str(int(x[2]  % 3600  //60)), 
                               str(int(x[2] % 60)).zfill(2))] for x in rf]
def extract_mah_stuff(media, segmented_stamps, outdir = None):
    timestamps = []
    nameswitch = False
    timestamps_ext = segmented_stamps#extract_music(segmentation)#[['{}:{}:{}'.format(str(int(x[1]//3600)), str(int(x[1] % 3600 //60)), str(int(x[1] % 60)).zfill(2)),  '{}:{}:{}'.format(str(int(x[2]//3600)), str(int(x[2]  % 3600  //60)), str(int(x[2] % 60)).zfill(2))] for x in r]
    try:
        for i in open(r"\\TIRIONFORDRING\ytd\timstamp.ini", 'r', encoding='UTF-8'):
            i = i.replace('\n','').replace('」', '').replace('~「',' ').replace('「', ' ').replace('『', ' ').replace('』', ' ')
            if ':' in i:
                timestamps.append([i[:i.find(' ')], i[i.find(' ')+1:]])
                #timestamps[-1][1] = timestamps[-1][1][1:]
                timestamps[-1][1] = timestamps[-1][1].replace('/', 'by')
                #timestamps[-1][1] = timestamps[-1][1].replace('-', 'by')
                while timestamps[-1][1][0] == ' ': timestamps[-1][1] = timestamps[-1][1][1:]
                while timestamps[-1][1][-1] == ' ': timestamps[-1][1] = timestamps[-1][1][:-1]
        #        nameswitch = True
            elif nameswitch:
                nameswitch = False
                timestamps[-1].append(i)
        if len(timestamps) > 0 and len(timestamps) != len(timestamps_ext): raise Exception('check timestamp assist', timestamps, timestamps_ext)
        with open(r"\\TIRIONFORDRING\ytd\timstamp.ini", 'w') as f:
            pass
    except FileNotFoundError:
        pass
    if len(timestamps) > 0: print('timestamp assist', timestamps)
    nameswitch = False
    file = media
    filename = file[:file.rfind('.')]
    fileext = file[len(filename):]
    filename = os.path.basename(filename)
    for i in range(len(timestamps_ext)):
        oud = outdir if outdir else os.path.dirname(file)
        #fileext = '.mp3'
        try:
            prefix = timestamps[i][1]
            subprocess.call('ffmpeg -i "{}" {} -c:v copy -c:a copy "{}"'.format(
            #subprocess.call('ffmpeg -i "{}" {} -vn -ab 320k "{}"'.format(
            file, 
            '-ss {} -to {}'.format(timestamps[i][0], timestamps_ext[i][1],),
            os.path.join(oud, filename + '_{}_{}'.format(str(i), prefix) + fileext)))
        except:
            prefix = str(i)
            subprocess.call('ffmpeg -i "{}" {} -c:v copy -c:a copy "{}"'.format(
            #subprocess.call('ffmpeg -i "{}" {} -vn -ab 320k "{}"'.format(
                file, 
                '-ss {} -to {}'.format(timestamps_ext[i][0], timestamps_ext[i][1],),
                os.path.join(oud, filename + '_' + prefix + fileext)
        ))

from ShazamAPI import Shazam
import time, json
def shazam(mp3, stop_at_first_match = True, force_sleep = 5):
    mp3_file_content_to_recognize = open(mp3, 'rb').read()
    shazam = Shazam(mp3_file_content_to_recognize)
    recognize_generator = shazam.recognizeSong()
    matches = []
    try:
        while True:
            match = next(recognize_generator)
            if len(match[1]['matches']) > 0: 
                matches.append(match)
                if stop_at_first_match: raise StopIteration()
    except StopIteration:
        pass
    return matches

def legalize_filename(fn):
    return fn.replace(':', ' ').replace('"', '').replace(r'/', '').replace(r'?', '').replace(r'*','')

def shazam_title(match):
    return [
    legalize_filename(match[1]['track']['title']),
    legalize_filename(match[1]['track']['subtitle']),    
    ]
results = {}

In [2]:
#GeForce840M爆内存
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1224)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs, 1 Logical GPUs


In [3]:
#单歌回切片
media = r"\\TIRIONFORDRING\ytd\[MUS1CA] 【#109】MUS1CA毎日歌配信 20220602.webm"
extract_mah_stuff(media, extract_music(segment(media, batch_size = 16)), outdir = r'\\TIRIONFORDRING\ytd\convert2music')

c:\pthon3.8.5\lib\site-packages\pyannote\algorithms\utils\viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
c:\pthon3.8.5\lib\site-packages\pyannote\algorithms\utils\viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
c:\pthon3.8.5\lib\site-packages\inaSpeechSegmenter\segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
c:\pthon3.8.5\lib\site-packages\numpy\core\_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


428/428 [==============================] - 9s 11ms/step


In [ ]:
#debug
segmented = segment(media)
print(extract_music(segmented))

In [ ]:
#全歌回切片
try:
    k = json.load(open(os.path.join(os.environ['temp'], 'ina.temp'), 'r'))
except:
    k = []
try:
    for media in glob.glob(r'\\MYSTERIOUSCHALL\Downloads\hedvika\*'):
        if media in k: continue
        print('extracting', media)
        segmentation=segment(media)
        extract_mah_stuff(media, extract_music(segmentation), outdir = r'D:\tmp\ytd\hedvika')
        k.append(media)
except:
    json.dump(k, open(os.path.join(os.environ['temp'], 'ina.temp'), 'w'))
    raise

In [22]:
#### SHAZAM识歌
for file in glob.glob(r'D:\tmp\ytd\hedvika\*'):
    filename = file[:file.rfind('.')]
    fileext = file[len(filename):]
    fn = os.path.basename(filename)
    if fn in results and results[fn] != None: print(fn, results[fn], 'loaded')
    elif not fn in results:
        print('shazaming', fn)
        try:
            results[fn] = shazam_title(shazam(file)[0])
            print(fn, 'shazam found to be', results[fn])
            os.rename(file, 
                 os.path.join(
                 #os.path.dirname(file),
                     r'D:\tmp\ytd\convert2music',
                (fn + "_{} by {}".format(results[fn][0].replace(':', ' '), results[fn][1].replace(r'/', ''))) + fileext
                 ))
        except IndexError:
            print(fn, 'shazam failed')
            results[fn] = None

shazaming [海德薇的录播组] 2月16日-唱歌学歌回（2） 20220207_5
[海德薇的录播组] 2月16日-唱歌学歌回（2） 20220207_5 shazam found to be ['Paprika', 'Kenshi Yonezu']
shazaming [海德薇的录播组] 2月16日-唱歌学歌回（2） 20220207_6
[海德薇的录播组] 2月16日-唱歌学歌回（2） 20220207_6 shazam found to be ['向かい風に打たれながら', '茅原実里']
shazaming [海德薇的录播组] 2月16日-唱歌学歌回（2） 20220207_7
[海德薇的录播组] 2月16日-唱歌学歌回（2） 20220207_7 shazam found to be ['ツギハギスタッカート', 'とあ']
shazaming [海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_0
[海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_0 shazam found to be ['海底', '凤凰传奇']
shazaming [海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_1
[海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_1 shazam found to be ['海底', '凤凰传奇']
shazaming [海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_10
[海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_10 shazam failed
shazaming [海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_11
[海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_11 shazam found to be ['No Doubt', 'OFFICIAL HIGE DANDISM']
shazaming [海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_2
[海德薇的录播组] 2月16日-唱歌学歌回（3） 20220207_2 shazam found to be ['海底', '凤凰传奇']
shazaming [海德薇的录播组] 2月16日-唱歌学歌回（3） 2

[海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_5 shazam found to be ['Awaken', 'League Of Legends, Valerie Broussard & Ray Chen']
shazaming [海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_6
[海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_6 shazam found to be ['Portal 2 Song', 'No No No Feat. Me']
shazaming [海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_7
[海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_7 shazam found to be ['Red Dahlia', 'Mili']
shazaming [海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_8
[海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_8 shazam found to be ['Into The Night (English Version)', 'YOASOBI']
shazaming [海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_9
[海德薇的录播组] 2月25日-唱歌之前记得吃饭（2） 20220207_9 shazam found to be ['優しい彗星', 'YOASOBI']
shazaming [海德薇的录播组] 2月25日-唱歌之前记得吃饭（3） 20220207_0
[海德薇的录播组] 2月25日-唱歌之前记得吃饭（3） 20220207_0 shazam found to be ['The Water', 'Hurts']
shazaming [海德薇的录播组] 2月25日-唱歌之前记得吃饭（3） 20220207_1
[海德薇的录播组] 2月25日-唱歌之前记得吃饭（3） 20220207_1 shazam found to be ['The Other Side Of Paradise', 'Glass Animals']
shazaming [海德薇的录播组] 2月25日-唱歌之前记得吃饭（3

[海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_1 shazam found to be ["Kimi No Hitomi Ni Koi Shiteru-Can't Take My Eyes Off Of You", 'Sheena Ringo']
shazaming [海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_2
[海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_2 shazam found to be ['女神 ~女神ソフィア討滅戦~', '祖堅正慶']
shazaming [海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_3
[海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_3 shazam found to be ['The Phantom of the Opera', 'Andrew Lloyd Webber']
shazaming [海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_4
[海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_4 shazam found to be ['Fire', 'Queen Bee']
shazaming [海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_5
[海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_5 shazam found to be ['Sad', 'X The Legend']
shazaming [海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_6
[海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_6 shazam found to be ['Last Surprise', 'Lyn']
shazaming [海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_7
[海德薇的录播组] 3月15日-好饿，先嗦个粉再唱歌（1） 20220301_7 shazam found to be ['The Other Side Of Paradise', 'Glass Animal

[海德薇的录播组] 3月1日-歌！（2） 20220301_2 shazam found to be ['Weight Of The World Kowaretasekainouta-Marina Kawano', 'Keiichi Okabe']
shazaming [海德薇的录播组] 3月1日-歌！（2） 20220301_3
[海德薇的录播组] 3月1日-歌！（2） 20220301_3 shazam found to be ['shadowgraph', 'MYTH & ROID']
shazaming [海德薇的录播组] 3月1日-歌！（2） 20220301_4
[海德薇的录播组] 3月1日-歌！（2） 20220301_4 shazam found to be ['FOREVER LOST', 'MYTH & ROID']
shazaming [海德薇的录播组] 3月1日-歌！（2） 20220301_5
[海德薇的录播组] 3月1日-歌！（2） 20220301_5 shazam found to be ['FOREVER LOST', 'MYTH & ROID']
shazaming [海德薇的录播组] 3月1日-歌！（2） 20220301_6
[海德薇的录播组] 3月1日-歌！（2） 20220301_6 shazam found to be ["Everything's Alright (From To the Moon)", 'Laura Shigihara']
shazaming [海德薇的录播组] 3月1日-歌！（2） 20220301_7
[海德薇的录播组] 3月1日-歌！（2） 20220301_7 shazam found to be ['Suicide', 'suisoh']
shazaming [海德薇的录播组] 3月1日-歌！（2） 20220301_8
[海德薇的录播组] 3月1日-歌！（2） 20220301_8 shazam found to be ['食虫植物', 'RIM']
shazaming [海德薇的录播组] 3月1日-歌！（2） 20220301_9
[海德薇的录播组] 3月1日-歌！（2） 20220301_9 shazam found to be ['悪魔の踊り方', 'キタニタツヤ']
shazami

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'D:\\tmp\\ytd\\hedvika\\[海德薇的录播组] 3月1日-歌！（3） 20220301_2.mp3' -> 'D:\\tmp\\ytd\\convert2music\\[海德薇的录播组] 3月1日-歌！（3） 20220301_2_When My N***a Died by CashFlow.mp3'